In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import nltk
nltk.download('stopwords')
nltk.download('punkt')


from functions import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [55]:
# if data not saved as csv, run this
import os
if not os.path.exists('data/merged_titles_labels.csv'):
    df1 = pd.read_csv('../eda/small1/labeled.csv')
    df2 = pd.read_csv('../eda/small2/labeled.csv')
    df3 = pd.read_csv('../eda/small3/labeled.csv')
    df = pd.concat([df1, df2, df3], ignore_index=True).reset_index(drop=True)
    df.to_csv('data/merged_titles_labels.csv', index=False)
    df.head()
else:
    df = pd.read_csv('data/merged_titles_labels.csv')
df.head()

,title,is_clickbait
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,Why the Truth Might Get You Fired,1
3,15 Civilians Killed In Single US Airstrike Hav...,1
4,Iranian woman jailed for fictional unpublished...,1


In [56]:
%run functions.py
if not os.path.exists('data/preprocessed_titles_labels.pkl'):
    df = preprocess_title(df)
    df.to_pickle('data/preprocessed_titles_labels.pkl') 
else:
    df = pd.read_pickle('data/preprocessed_titles_labels.pkl')
df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,title,is_clickbait
0,"[house, aide, even, see, letter, jason]",1
1,"[hillary, clinton, big, woman, campus]",0
2,"[truth, might, get, fired]",1
3,"[fifteen, civilian, single, u]",1
4,"[iranian, woman, fictional, unpublished, story...",1


In [57]:
from tqdm.notebook import tqdm
import time
# VECTOR_SIZE = 2500
# WINDOW = 4
EPOCHS = 500

# EPOCHS = 100
WORKERS = 16
MIN_COUNT = 1
# train many word2vec models with diferent VECTOR_SIZE and WINDOW
VECTOR_SIZEs = [
    100, 
    250, 
    500, 
    1000, 
    # 1500, 
    # 2000, 
    # 2500, 
    # 3000
]

WINDOWs = [3, 4, 5, 6, 7, 8]
SGs = [0, 1]


##################################################
# # --uncomment for sample model training--
# EPOCHS = 200
# VECTOR_SIZEs = [500]
# WINDOWs = [4]
# SGs = [0]
##################################################


print('Start training')
# sleep 200 ms
time.sleep(0.2)

for VECTOR_SIZE in tqdm(VECTOR_SIZEs):
    print(f'Current VECTOR_SIZE: {VECTOR_SIZE}')
    for WINDOW in tqdm(WINDOWs, desc=f'WINDOW'):
        for sg in tqdm(SGs, desc=f'SG'):
            model = Word2Vec(df['title'], vector_size=VECTOR_SIZE, window=WINDOW, min_count=MIN_COUNT, workers=WORKERS, sg=sg)
            model.train(df['title'], total_examples=len(df['title']), epochs=EPOCHS)
            model.save(f'word2vec_models/word2vec_vs{VECTOR_SIZE}_win{WINDOW}_sg{sg}.model')

Start training


  0%|          | 0/4 [00:00<?, ?it/s]

Current VECTOR_SIZE: 100


WINDOW:   0%|          | 0/6 [00:00<?, ?it/s]

SG:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model.wv.most_similar('trump')

[('u', 0.5647323131561279),
 ('new', 0.5609492659568787),
 ('hillary', 0.5219982862472534),
 ('time', 0.517253577709198),
 ('clinton', 0.47899648547172546),
 ('donald', 0.4427263140678406),
 ('york', 0.4189266264438629),
 ('say', 0.4157974123954773),
 ('house', 0.3809012770652771),
 ('one', 0.3720136880874634)]